# Workshop: Pollen recognition using Convolutional Neuronal Networks. First steps in keras 

## Creating a model

1. Debug here. 
2. When is ready move it to a python script. 

### Sequential 

During this workshop we will study only sequential models.There are other type of models that support more complicated and advanced architectures. 

The steps to create sequential models is very simple: 

*  Initialize your model by calling the class model Sequential()
*  Add layers depending the functionality you want. 
    * Dense layers
    * Convolutional Layer
    * MaxPooling 
    * Flatten 
*  Make sure input dimensions are correct. 
    * Input dimensions come from the size of the input images. For the convolutional layer you need to specify
    the dimensions and the channels in an specific order. Using theano backend "channels first". Using tensorflow backend "channels last"
    * Check also the output. For binary classification usually the sumarizing layer only would have 2 units. 
    
*  Compile your model

*  Fit and evaluate

```python
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger, TensorBoard
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution3D
from keras.layers.convolutional import Conv2D
from sklearn.preprocessing import scale


def logistic_regresor(units=1,input_dim=32*32*3,
						 activation='sigmoid', loss='binary_crossentropy',
						 				optimizer='sgd',metrics = 'accuracy'):
	model = Sequential()
	model.add(Dense(units, input_dim=input_dim, activation=activation))

	model.compile(loss=loss,
              optimizer=optimizer,
              metrics=[metrics])
	return model 

def shallow_model(input_shape=(3,90,150), lr =0.001, kernels=16, stride=(5,5),pool_size=(2,2), dense=50):
	model = Sequential()

	model.add(Conv2D(kernels, stride, input_shape=input_shape,data_format="channels_first"))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=pool_size))

	model.add(Flatten())
	model.add(Dense(dense, activation='relu'))
	model.add(Dense(2, activation='softmax'))
	model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=lr),
              metrics=['accuracy'])
	return model

def two_layer_model(input_shape=(3,100,100), lr =0.001, kernels=16, stride=(13,13),pool_size=(2,2), dense=50):
	model = Sequential()
	model.add(Conv2D(kernels, stride, input_shape=input_shape))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=pool_size))
	model.add(Conv2D(kernels, stride, input_shape=input_shape))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=pool_size))
	model.add(Flatten())
	model.add(Dense(dense, activation='relu'))
	model.add(Dense(2, activation='softmax'))
	model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=lr),
              metrics=['accuracy'])
	return model
````

# checkpoints  

Training Neuronal Networks  can take long and might envolve some errors or issues on the way. Is always a good practice, for instance, save the weights often, save the logs according to the metrics used, and stop early if the training is not improving learning.  Gladly for us,  Keras make it very simple!  The keras  callbacks are design to call out functions with some periodicity to check how the network is doing. Let's check some of them

* **ModelCheckpoint**: 

This call backs let us save the weights in some specific location, according to the periodicity we want or the metric we are using. 

* **CSVLogger**: We can also save all the logs obtained in an indicated location in csv format. 

* **TensorBoard**: This tool creates also logs, but it also includes an interface for visualization.



```python
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger, TensorBoard

WEIGHTS_BEST = 'weights/weights.best.h5'
TRAINING_LOG = 'logs/training_log,csv'
LOGS_DIR = './logs'


checkpoint = ModelCheckpoint(WEIGHTS_BEST, monitor='loss', verbose=0, save_best_only=False, save_weights_only=True, mode='min', period=1) 
csv_logger = CSVLogger(TRAINING_LOG, append=True)
tb = TensorBoard(log_dir=LOGS_DIR, histogram_freq=0, write_graph=True, write_images=False)


callbacks_list = [ checkpoint, csv_logger, tb]
```

# Launching your model

Once you have your model, you can star training. To do so you can call the model.fit(*parameters*) method. It has the following inputs

**Data, target **: Data for training and labels. More complicated models not only accept labels but could accept ground_truth images, such as heatmaps. Also this can be replaced by a generator that can be used for augmenting data. 

**batch Size:** How many images are you feeding on each step.

**epochs:** The maximum number of epocs that you want to train your model. 

**validation data:** You might want to save the performance of your model while is training, so you can put your validation data to check the performance in your testing dataset. 

**callbacks:** see previous section

After you finish training you can evaluate your model by providing testing dataset. You can specify what kind of metrics you can use such as F-1 score, accuracy, etc. 


#### Logistic Regressor 

```python 
batch_size = 6
max_iter= 200 
model=logistic_regresor()
history = model.fit(X_train,y_train,
                    batch_size = batch_size,
                    epochs=max_iter,
                    validation_data=(X_test,y_test),
                    callbacks=callbacks_list)
results = model.evaluate(self.test,self.test_y)
H_logistic = pd.DataFrame(history.history, index=history.epoch)
```

#### One layer Shallow Model

```python 
batch_size = 6
max_iter= 200 
model=shallow_model()
history = model.fit(X_train,y_train,
                    batch_size = batch_size,
                    epochs=max_iter,
                    validation_data=(X_test,y_test),
                    callbacks=callbacks_list)
results = model.evaluate(self.test,self.test_y)
H_logistic = pd.DataFrame(history.history, index=history.epoch)
```

#### Two Layer Shallow Model 

```python 
batch_size = 6
max_iter= 200 
model=two_layer_model()
history = model.fit(X_train,y_train,
                    batch_size = batch_size,
                    epochs=max_iter,
                    validation_data=(X_test,y_test),
                    callbacks=callbacks_list)
results = model.evaluate(self.test,self.test_y)
H_logistic = pd.DataFrame(history.history, index=history.epoch)
```

# Visualizing the results

Visualizing the results is a crucial to being able to say how good a model is working. 
There are basically two visualizations we will try to look at today.  

1. Visualize the metrics, losses, logs... 

2. Visualize the confusion matrix. 

### Visualize the Metrics

To visualize the metrics we can use the help of pandas and matplotlib

# Writting the scripts

We are ready to write the scripts! Now that we know that the models are working and that input dimensions and architecture of the networks are compatible, we will put the different parts of the pipeline in python scripts and then we will create a bash script to run in Bridges. 

# Bridges 

It is important to make clear some important facts for running jobs on bridges. For future interaction with the server, please read carefully the [documentation](https://portal.xsede.org/psc-bridges).  Every job on bridges has some cost, so it is important to make sure to put the right options on the scripts so we don't waste resources. The pricing is based on the cores used. For this workshop we will used gpu-shared space. Using gpu-shared space is cheaper and usually these jobs get allocated faster. 

To start an interactive session type the following command: 

```bash
interact -gpu 
```